In [7]:
import numpy as np
from scipy.special import digamma as dga
from scipy.special import gamma as ga
from scipy.special import loggamma as lga

In [8]:
eps=1e-10
def log(x):
    return np.log(x + eps)

def digamma(x):
    return dga(x + eps)

def loggamma(x):
    return lga(x + eps)

In [9]:
def init(data):
    vocab = np.array([i for i in range(100)])

    num_doc = data.shape[0]
    num_vocab = vocab.shape[0]
    len_doc = data.shape[1]
    num_topic = 10

    w = np.zeros([num_doc, len_doc, num_vocab])
    for d in range(num_doc):
        for n in range(len_doc):
            w[d, n, data[d, n]] = 1

    alpha = np.ones(shape=num_topic)
    eta = np.ones(shape=num_vocab)

    phi = np.random.rand(num_doc, len_doc, num_topic)
    for d in range(num_doc):
        for n in range(len_doc):
            phi[d, n] /= np.sum(phi[d, n])

    gam = np.random.rand(num_doc, num_topic)
    gam /= np.sum(gam, axis=1)[:, np.newaxis]

    lam = np.random.rand(num_topic, num_vocab)
    lam /= np.sum(lam, axis=1)[:, np.newaxis]
    return lam, gam, phi, w, num_doc, num_topic, num_vocab, len_doc, alpha, eta

In [10]:
def one_step(lam, gam, phi, w, num_doc, num_topic, num_vocab, len_doc, alpha, eta):
    #print(num_doc, num_topic, num_vocab)
    for k in range(num_topic):
        lam[k] = eta
        for d in range(num_doc):
            for n in range(len_doc):
                lam[k] += phi[d, n, k] * w[d, n]
    #lam /= np.sum(lam, axis=1)[:, np.newaxis]
    
    gam = alpha + np.sum(phi, axis=1)
    #gam /= np.sum(gam, axis=1)[:, np.newaxis]
    
    def get_single_doc(lam, gam, phi, w, d):
        for n in range(len_doc):
            #phi[d, n, :] = np.exp(digamma(gam[d, :]) + digamma(lam[:, data[d, n]]) - digamma(np.sum(lam, axis=1)))
            for k in range(num_topic):
                phi[d, n, k] = np.exp(digamma(lam[k, data[d, n]]) - digamma(np.sum(lam[k])) + digamma(gam[d, k]) - digamma(np.sum(gam[d])))
            phi[d, n, :] /= np.sum(phi[d, n, :])
        return phi[d], d
            
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        future_list = [executor.submit(get_single_doc, lam, gam, phi, w, d) for d in range(num_doc)]
        for future in concurrent.futures.as_completed(future_list):
            phi_d, d = future.result()
            phi[d] = phi_d
    
    return lam, gam, phi, w, num_doc, num_topic, num_vocab, len_doc, alpha, eta

In [11]:
import concurrent.futures

def get_res1(lam, gam, phi, w):
    res_1 = 0.0
    res_1 += num_topic * loggamma(np.sum(eta))
    res_1 -= num_topic * np.sum(loggamma(eta))
    '''
    for k in range(num_topic):
        for i in range(num_vocab):
            res_1 += (eta[i] - 1) * (digamma(lam[k, i]) - digamma(np.sum(lam[k])))
    '''
    return res_1


def get_res2(lam, gam, phi, w):          
    res_2 = 0.0
    for n in range(len_doc):
        for k in range(num_topic):
            res_2 += phi[:, n, k] * (digamma(gam[:, k]) - digamma(np.sum(gam, axis=1)))
    #res_2 -= digamma(np.sum(gam, axis=1))
    res_2 = np.sum(res_2)
    return res_2

    
def get_res3(lam, gam, phi, w):
    res_3 = 0.0
    res_3 += loggamma(np.sum(alpha))
    res_3 -= np.sum(loggamma(alpha))
    '''
    for k in range(num_topic):
        res_3 += (alpha[k] - 1) * (digamma(gam[:, k] - digamma(np.sum(gam[:, k]))))
    '''
    res_3 = np.sum(res_3)
    return res_3

def get_res4(lam, gam, phi, w):
    res_4 = 0.0
    def get_res4_single_loc(lam, gam, phi, w, n):
        res_loc = 0.0
        for k in range(num_topic):
            sum_lam_k = np.sum(lam[k])
            for i in range(num_vocab):
                res_loc += phi[:, n, k] * w[:, n, i] * (digamma(lam[k, i]) - digamma(sum_lam_k))
        res_loc = np.sum(res_loc)
        return res_loc
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=32) as executor:
        future_list = [executor.submit(get_res4_single_loc, lam, gam, phi, w, n) for n in range(len_doc)]
        for future in concurrent.futures.as_completed(future_list):
            res_4 += future.result()
    return res_4

def get_res5(lam, gam, phi, w):
    res_5 = 0.0
    for k in range(num_topic):
        res_5 += loggamma(np.sum(lam[k])) - np.sum(loggamma(lam[k]))
    for k in range(num_topic):
        sum_lam_k = np.sum(lam[k])
        #'''
        res_5 += np.sum((lam[k] - 1) * (digamma(lam[k]) - digamma(sum_lam_k)))
        '''
        for i in range(num_vocab):
            res_5 += (lam[k, i] - 1) * (digamma(lam[k, i]) - digamma(sum_lam_k))
        '''
    return -res_5
    
def get_res6(lam, gam, phi, w):
    res_6 = 0.0
    res_6 += np.sum(phi * log(phi))
    return -res_6

def get_res7(lam, gam, phi, w):
    res_7 = 0.0
    res_7 += loggamma(np.sum(gam, axis=1)) - np.sum(loggamma(gam), axis=1)
    #print(res_7)
    res_7 = np.sum(res_7)
    for d in range(num_doc):
        res_7 += np.sum((gam[d] - 1) * (digamma(gam[d]) - digamma(np.sum(gam[d]))))
    return -res_7

def elbo(lam, gam, phi, w):
    res = 0.0
    func_list = [get_res1, get_res2, get_res3, get_res4, get_res5, get_res6, get_res7]
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        future_list = [executor.submit(func, lam, gam, phi, w) for func in func_list]
        for future in concurrent.futures.as_completed(future_list):
            res += future.result()
    
    return res

In [ ]:
elbo_list = []
#for dsize in range(10, 100, 10):
    #data = np.load("mcs_hw4_p1_lda.npy")[:dsize]
if True:
    data = np.load("mcs_hw4_p1_lda.npy")
    for i in range(10):
        lam, gam, phi, w, num_doc, num_topic, num_vocab, len_doc, alpha, eta = init(data)
        lam, gam, phi, w, num_doc, num_topic, num_vocab, len_doc, alpha, eta = one_step(lam, gam, phi, w, num_doc, num_topic, num_vocab, len_doc, alpha, eta)
        print("iteration " + str(i) + " done")
        elbo_per_point = elbo(lam, gam, phi, w) / dsize
        elbo_list.append(elbo_per_point)
        print(elbo_per_point)

iteration 0 done
-104672.74536964955
iteration 1 done
-104672.74416709045
iteration 2 done
-104672.743821647
iteration 3 done
-104672.74071140107
iteration 4 done
-104672.74362164483
iteration 5 done
-104672.75411491861
iteration 6 done
-104672.74591432579
iteration 7 done
-104672.74464830024
iteration 8 done
-104672.7422740624


In [52]:
r = []
r.append(get_res1(lam, gam, phi, w))
r.append(get_res2(lam, gam, phi, w))
r.append(get_res3(lam, gam, phi, w))
r.append(get_res4(lam, gam, phi, w))
r.append(get_res5(lam, gam, phi, w))
r.append(get_res6(lam, gam, phi, w))
r.append(get_res7(lam, gam, phi, w))

In [13]:
data = np.load("mcs_hw4_p1_lda.npy")
lam, gam, phi, w, num_doc, num_topic, num_vocab, len_doc, alpha, eta = init(data)
np.sum(phi, axis=1).shape

(10000, 10)

In [14]:
alpha.shape

(10,)

In [15]:
np.sum(phi, axis=1)

array([[21.15514924, 21.43316276, 20.01395873, ..., 20.92318003,
        19.99547232, 20.19644538],
       [20.91252212, 18.80776031, 21.48994937, ..., 21.03521142,
        18.81068813, 20.32623107],
       [18.92401821, 21.93323691, 18.91930827, ..., 20.81140416,
        20.59745631, 19.49480417],
       ...,
       [21.06326835, 20.61386613, 20.6810978 , ..., 18.51251105,
        19.3217346 , 20.34843406],
       [20.62657111, 20.82458292, 20.44146343, ..., 20.08851118,
        19.11378451, 20.38167739],
       [20.55558543, 20.45527328, 20.64976988, ..., 18.09921177,
        19.98834929, 20.59680971]])

In [16]:
np.sum(phi, axis=1) + alpha

array([[22.15514924, 22.43316276, 21.01395873, ..., 21.92318003,
        20.99547232, 21.19644538],
       [21.91252212, 19.80776031, 22.48994937, ..., 22.03521142,
        19.81068813, 21.32623107],
       [19.92401821, 22.93323691, 19.91930827, ..., 21.81140416,
        21.59745631, 20.49480417],
       ...,
       [22.06326835, 21.61386613, 21.6810978 , ..., 19.51251105,
        20.3217346 , 21.34843406],
       [21.62657111, 21.82458292, 21.44146343, ..., 21.08851118,
        20.11378451, 21.38167739],
       [21.55558543, 21.45527328, 21.64976988, ..., 19.09921177,
        20.98834929, 21.59680971]])